<font size=6><b>Lec08.NLP 언어모델
* NLP (Natural Language Processing, 자연어 처리)   * 언어 모델(Language Model, LM)은 언어라는 현상을 모델링하고자 단어 시퀀스(문장)에 확률을 할당(assign)하는 모델 


<font size=6><b>언어모델
* N-gram
* <b>빈도기반 
    * <b>BoW(Bag of Words)
    * <b>TF-IDF(Term Frequency - Inverse Document Frequency)
* 코사인 유사도(Cosine Similarity)

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


sns.set()

#-------------------- 차트 관련 속성 (한글처리, 그리드) ---------ㅊ--
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
# from IPython.core.display import display, HTML
from IPython.display import display, HTML

display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

import warnings
warnings.filterwarnings(action='ignore')

# N-gram

In [2]:
text = 'Hello'
 
for i in range(len(text) - 1):             # 2-gram이므로 문자열의 끝에서 한 글자 앞까지만 반복함
    print(text[i], text[i + 1], sep='')    # 현재 문자와 그다음 문자 출력

He
el
ll
lo


In [3]:
text = 'this is python script'
words = text.split()                 # 공백을 기준으로 문자열을 분리하여 리스트로 만듦
 
for i in range(len(words) - 1):      # 2-gram이므로 리스트의 마지막에서 요소 한 개 앞까지만 반복함
    print(words[i], words[i + 1])    # 현재 문자열과 그다음 문자열 출력

this is
is python
python script


# BoW(Bag of Words)
* 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
* 문장사이즈만큼 단일행렬만든다[1,2,0,1,1] >> 희소행렬의 문제를 해결하기위해 [[],[],[]]

In [4]:
# doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords


corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

# stop_words_list = stopwords.words('english')
# print('불용어 개수 :', len(stop_words_list))
# print('불용어 10개 출력 :',stop_words_list[:10])

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray()) 

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)    # 임베딩(숫자 - 단어)한다


bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [5]:
vocab_sorted = sorted(vector.vocabulary_, key = lambda x:x[1], reverse = True)
print(vocab_sorted)

['you', 'your', 'love', 'know', 'because', 'want']


# DTM
문서 단어 행렬(DTM, Document-Term Matrix)
* 각 문서에서 등장한 단어의 빈도를 행렬의 값으로 표기해 서로 다른 문서들을 비교
* 다수의 BoW : 각 문서에 대한 BoW를 하나의 행렬로 만든 것

* <단점>
    - 희소 표현(Sparse representation) : 단어 집합의 크기 == 벡터의 차원이 되고 대부분의 값이 0이 된다
    - "단순" 빈도 수 기반 접근 : '이다'와 같은 중요하지 않은 최빈도 단어로 문서를 연관지으면??
    - <font color='red'> TF-IDF 필요 (DTM에 불용어와 중요한 단어에 대해서 가중치)</font>

### sklearn.CountVectorizer()

In [6]:
corpus = ['먹고 싶은 사과',
        '먹고 싶은 바나나',
        '길고 노란 바나나 바나나',
        '저는 과일이 좋아요']

In [7]:
vector = CountVectorizer()
print('bag of words vector : \n', vector.fit_transform(corpus).toarray()) 
print('vocabulary :\n',vector.vocabulary_) # 순서 뒤죽박죽
#--------단어정렬
vocab_sorted = sorted(vector.vocabulary_.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

vocab_word = dict(vocab_sorted).keys()
print(vocab_word)



# ------ DTM
vec = vector.fit_transform(corpus).toarray()
df=pd.DataFrame(vec, columns=(vocab_word))
print(vector.vocabulary_)
df.head() 

bag of words vector : 
 [[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
vocabulary :
 {'먹고': 3, '싶은': 6, '사과': 5, '바나나': 4, '길고': 1, '노란': 2, '저는': 7, '과일이': 0, '좋아요': 8}
[('좋아요', 8), ('저는', 7), ('싶은', 6), ('사과', 5), ('바나나', 4), ('먹고', 3), ('노란', 2), ('길고', 1), ('과일이', 0)]
dict_keys(['좋아요', '저는', '싶은', '사과', '바나나', '먹고', '노란', '길고', '과일이'])
{'먹고': 3, '싶은': 6, '사과': 5, '바나나': 4, '길고': 1, '노란': 2, '저는': 7, '과일이': 0, '좋아요': 8}


,좋아요,저는,싶은,사과,바나나,먹고,노란,길고,과일이
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


### sklearn. TfidfVectorizer()
* from sklearn.feature_extraction.text import TfidfVectorizer
* (IDF의 로그항의 분자에 1을 더해주며, 로그항에 1을 더해주고, TF-IDF에 L2 정규화라는 방법으로 값을 조정

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus_list = ['먹고 싶은 사과',
        '먹고 싶은 바나나',
        '길고 노란 바나나 바나나',
        '저는 과일이 좋아요']

tfidfv = TfidfVectorizer()
# TF-IDF 계산
tfidf_arr = tfidfv.fit_transform(corpus_list).toarray()
print(tfidf_arr)

# vocab = ['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']
vocab = tfidfv.vocabulary_
print()  

df = pd.DataFrame(tfidf_arr, columns=vocab)
df

[[0.         0.         0.         0.52640543 0.         0.66767854
  0.52640543 0.         0.        ]
 [0.         0.         0.         0.57735027 0.57735027 0.
  0.57735027 0.         0.        ]
 [0.         0.47212003 0.47212003 0.         0.7444497  0.
  0.         0.         0.        ]
 [0.57735027 0.         0.         0.         0.         0.
  0.         0.57735027 0.57735027]]



,먹고,싶은,사과,바나나,길고,노란,저는,과일이,좋아요
0,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
2,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
3,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


# TF_IDF
* 단어 빈도-역 문서 빈도(TF-IDF, Term Frequency-Inverse Document Frequency)
* 우선 DTM을 만든 후, TF-IDF 가중치를 부여
* 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 활용
* TF-IDF = TF * IDF

* 문서(d), 단어(t), 문서총개수(n)
* <font color='red'> $tf(d,t)$ : 특정 문서 d에서의 특정 단어 t의 등장 횟수</font> →
* <font color='red'>  $df(t)$ : 특정 단어 t가 등장한 문서의 수 </font>  ↓
* <font color='red'>  $idf(d, t) = log (\frac{n}{1+df(t)})$ : df(t) 역수</font>

## [실습]

In [8]:
corpus = ['먹고 싶은 사과',
        '먹고 싶은 바나나',
        '길고 노란 바나나 바나나',
        '저는 과일이 좋아요']
vocab = ['좋아요', '저는', '싶은', '사과', '바나나', '먹고', '노란', '길고', '과일이']

## 공통함수

In [9]:
def tf(t, d): #DTM(BOW집합)
    return d.count(t)

def idf(t):
    df = 0
    for doc in corpus:
        df += t in doc
        # t(해당단어)의 빈도 누적 합
    return np.log(N/(df + 1))
#return np.log1p(N/df)

def tfidf(t, d):
    return tf(t,d)* idf(t)

## TF (TDM)
* 특정 문서에서 단어의 등장 횟수

In [10]:
N = len(corpus) #문서의 갯수

count_list_tf = []
for d in corpus:    # d : 문서              
    for t in vocab:  # vocab size :9 
        count_list_tf.append( tf(t,d) )
        
count_list_tf = np.array(count_list_tf).reshape(N,-1)  # TF
count_list_tf

array([[0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1]])

## IDF
* (단어가 등장한 문서의 수)의 역수

In [11]:
count_list_idf = []
for i in vocab:
    count_list_idf.append(idf(t))
count_list_idf
# 1/np.log1p(빈도수)  로그스케일링 해서

[0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453]

## TF-IDF

In [12]:
count_list_tf1 = np.array(count_list_tf)

In [13]:
type(count_list_tf1)

numpy.ndarray

In [14]:
count_list_idf1 =np.array(count_list_idf)

In [15]:
tf_idf = count_list_tf1 * count_list_idf1 

In [16]:
df = pd.DataFrame(tf_idf, columns=vocab)
df

,좋아요,저는,싶은,사과,바나나,먹고,노란,길고,과일이
0,0.000000,0.000000,0.693147,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.693147,0.000000,0.693147,0.693147,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,1.386294,0.000000,0.693147,0.693147,0.000000
3,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147


In [17]:
#정수 array와 실수 array는 브로드캐스팅이 안된다

## [실습]

### sklearn.CountVectorizer()

### sklearn. TfidfVectorizer()
* from sklearn.feature_extraction.text import TfidfVectorizer
* (IDF의 로그항의 분자에 1을 더해주며, 로그항에 1을 더해주고, TF-IDF에 L2 정규화라는 방법으로 값을 조정

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do '
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab= tfidfv.vocabulary_
print(tfidfv.transform(corpus).toarray())
print(vocab)


[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


# 코사인 유사도(Cosine Similarity)
* 벡터의 크기가 아니라 방향(패턴)에 초점
* 두 문서(문장) 간의 유사한 관계 정도를 나타냄 >> 문서간 관계에는 음수관계 없다

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbpLIxd%2Fbtq03F9AfHS%2FRTzzEkvHWvqQ5XfQh6rkY1%2Fimg.png"><br>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbayZ5E%2Fbtq04KWSnlM%2FAkYrOmeom4jOCLv2fBf1HK%2Fimg.png">

In [41]:
# |A| >> 노름 (규제 : 제곱합의 제곱근)

## 공통함수

In [20]:
A = df.loc[0].values
B = df.loc[1].values
# A, B
res1 = np.dot(A, B)
res2 = np.sqrt(np.sum(A**2)) * np.sqrt(np.sum(B**2))
cs = res1 / res2
print(f"코사인유사도 : {cs:.4f}")

# np.sqrt(np.sum(B**2)) 계산 원리 -------------
# B_sum = 0
# for b in B:
#     B_sum +=  b**2
# np.sqrt(B_sum)
# 제곱합의 제곱근 >> 노름

코사인유사도 : 0.6667


In [21]:
def my_cos_sim(A,B):  
    cos_sim = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
    print(cos_sim)   
my_cos_sim(A,B)

0.6666666666666666


In [22]:
import numpy as np
from numpy.linalg import norm  #--정규화 ||A||
#from numpy import dot

def my_cos_sim(A,B):
    cos_sim = np.dot(A, B) / (norm(A) * norm(B))
    print(cos_sim)   
    
# sklearn...vectorize.. [1,2,3,]
doc1 = [0,1,2,1]
doc2 = [1,0,1,1]
doc3 = [2,0,2,2]
my_cos_sim(doc1,doc2)
my_cos_sim(doc1,doc3)
my_cos_sim(doc2,doc3)    

0.7071067811865476
0.7071067811865476
1.0000000000000002


## 사용예

In [23]:
text_list = ["어제 비가 와서 날씨가 추워졌다.",
             "나는 고양이를 좋아한다.",
             "나는 강아지를 좋아한다."]


In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

### Tokenizer()

In [25]:
token = Tokenizer()
token.fit_on_texts(text_list)
text_seq_list = token.texts_to_sequences(text_list)
vocab = token.word_index
print(text_seq_list, vocab)

max_length = max(len(sent.split()) for sent in text_list )
print()
print('최대길이(Voca Size):', max_length)

padding_res = pad_sequences(sequences=text_seq_list,   #[[ 1, 2, ..] , ['1, 2, ..]]
                   maxlen=5,      #voca_size :문장들이 가지고 있는 단어갯수 중 최대 단어수
                   padding="pre"  #앞에서부터 0을 채워서 제일 긴 길이에 맞추기 pre/post
                  )
print()
print(padding_res)

cs1 = my_cos_sim(padding_res[0], padding_res[1])
cs2 = my_cos_sim(padding_res[0], padding_res[2])
cs3 = my_cos_sim(padding_res[1], padding_res[2])
print()
print(cs1, cs2, cs3)

[[3, 4, 5, 6, 7], [1, 8, 2], [1, 9, 2]] {'나는': 1, '좋아한다': 2, '어제': 3, '비가': 4, '와서': 5, '날씨가': 6, '추워졌다': 7, '고양이를': 8, '강아지를': 9}

최대길이(Voca Size): 5

[[3 4 5 6 7]
 [0 0 1 8 2]
 [0 0 1 9 2]]
0.6941980239122336
0.6774960231074738
0.9995786102379159

None None None


### CountVectorizer()

In [26]:
vector = CountVectorizer()
print()
vocab = vector.fit_transform(text_list).toarray()
print(vocab ) 
print()
print(vector.vocabulary_)



[[0 0 0 1 1 1 1 0 1]
 [0 1 1 0 0 0 0 1 0]
 [1 0 1 0 0 0 0 1 0]]

{'어제': 5, '비가': 4, '와서': 6, '날씨가': 3, '추워졌다': 8, '나는': 2, '고양이를': 1, '좋아한다': 7, '강아지를': 0}


In [27]:
print()
cs1 = my_cos_sim(vocab[0], vocab[1])
cs2 = my_cos_sim(vocab[0], vocab[2])
cs3 = my_cos_sim(vocab[1], vocab[2])
print()
print(cs1, cs2, cs3)


0.0
0.0
0.6666666666666667

None None None


### TF_IDF

In [44]:
# tf_idf
tfidf =TfidfVectorizer()
tf_idf = tfidf.fit_transform(text_list).toarray()

cs1 = my_cos_sim(tf_idf[0], tf_idf[1])
cs2 = my_cos_sim(tf_idf[0], tf_idf[2])
cs3 = my_cos_sim(tf_idf[1], tf_idf[2])
print()
print(cs1, cs2, cs3)

0.0
0.0
0.5363499141045837

None None None


# [실습] 영화 추천 시스템

In [29]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
stop_words_list = stopwords.words('english')
# print('불용어 개수 :', len(stop_words_list))
# print('불용어 10개 출력 :',stop_words_list[:10])

In [30]:
train = pd.read_csv("./datasets/movies_metadata.csv")[:20000]
# train.info()
data = pd.DataFrame(train)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  20000 non-null  object 
 1   belongs_to_collection  2399 non-null   object 
 2   budget                 20000 non-null  object 
 3   genres                 20000 non-null  object 
 4   homepage               3055 non-null   object 
 5   id                     20000 non-null  object 
 6   imdb_id                19993 non-null  object 
 7   original_language      19999 non-null  object 
 8   original_title         20000 non-null  object 
 9   overview               19865 non-null  object 
 10  popularity             19998 non-null  object 
 11  poster_path            19907 non-null  object 
 12  production_companies   19999 non-null  object 
 13  production_countries   19999 non-null  object 
 14  release_date           19983 non-null  object 
 15  re

## 결측처리(Overview)

In [31]:
print(data["overview"].isna().sum())
data['overview'] = data['overview'].fillna('')
print(data["overview"].isna().sum())


135
0


## 코사인 유사도

In [32]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)
# print(tfidf.vocabulary_)
print(tfidf_matrix[0].toarray())

TF-IDF 행렬의 크기(shape) : (20000, 47487)
[[0. 0. 0. ... 0. 0. 0.]]


In [33]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [34]:
cosine_sim[4]

array([0.        , 0.        , 0.02500492, ..., 0.        , 0.        ,
       0.        ])

In [35]:
df_cs = pd.DataFrame(cosine_sim,columns = data["title"], index =data["title"] )  # 사실은 overview - overview 의 유사도데이터지만, "영화"의 overview 라서 타이틀 적었다
df_cs.head()

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,The American President,Dracula: Dead and Loving It,Balto,Nixon,Cutthroat Island,Casino,Sense and Sensibility,Four Rooms,Ace Ventura: When Nature Calls,Money Train,Get Shorty,Copycat,Assassins,Powder,Leaving Las Vegas,Othello,Now and Then,Persuasion,The City of Lost Children,Shanghai Triad,Dangerous Minds,Twelve Monkeys,Wings of Courage,Babe,Carrington,Dead Man Walking,Across the Sea of Time,It Takes Two,Clueless,"Cry, the Beloved Country",Richard III,Dead Presidents,Restoration,Mortal Kombat,To Die For,How To Make An American Quilt,Se7en,Pocahontas,When Night Is Falling,The Usual Suspects,...,Kiss and Tell,"Oui, mais...",Space Adventure Cobra,L'aventure c'est l'aventure,A Season for Miracles,The Awful Truth,1911,Looking for Jackie,The Shaolin Temple,Shaolin Temple 2: Kids from Shaolin,Carol Channing: Larger Than Life,Memory,Martial Arts of Shaolin,Snowmageddon,Stand Up and Fight,Jazz,Barbara,A Liar's Autobiography: The Untrue Story of Monty Python's Graham Chapman,Little Miss Broadway,Miss Annie Rooney,Mr. Belvedere Goes to College,The Hobbit: An Unexpected Journey,Mon Paradis - Der Winterpalast,No Flesh Shall Be Spared,I am Von Höfler Variation on Werther,Born to Defend,Dragon Fight,The Master,Kung Fu Cult Master,The Fitzgerald Family Christmas,Hyde Park on Hudson,Lay the Favorite,Audition/Talent Competition,All's Faire in Love,Crooked Arrows,The Enforcer,Dr. Wai in the Scriptures with No Words,Once Upon a Time in China and America,Contract Killer,The Executor,Never Say... Never!,Calmos,How to Make Love to a Woman,"After Fall, Winter",Violeta Went to Heaven,Rebellion,Versailles,Two in the Wave,Lotte Reiniger: Homage to the Inventor of the Silhouette Film,"RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'"
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.015757,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.041140,0.000000,0.0,0.009941,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.019831,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.006348,0.000000,0.0,0.009321,0.00000,0.000000,0.000000,0.013822,...,0.000000,0.0,0.011365,0.0,0.000000,0.0,0.011791,0.014688,0.013539,0.007355,0.011161,0.000000,0.01452,0.000000,0.000000,0.0,0.000000,0.000000,0.019853,0.027106,0.00000,0.000000,0.0,0.0,0.000000,0.00000,0.117982,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.009979,0.060997,0.000000,0.0,0.0,0.024585,0.0,0.0,0.0
Jumanji,0.015757,1.000000,0.049073,0.0,0.000000,0.051830,0.00000,0.000000,0.106355,0.0,0.007616,0.0,0.0,0.008233,0.0,0.0,0.000000,0.028753,0.000000,0.0,0.000000,0.0,0.006043,0.000000,0.0,0.024285,0.024941,0.006489,0.0,0.000000,0.000000,0.047250,0.0,0.005560,0.004326,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.025758,0.0,0.000000,0.01848,0.008127,0.000000,0.016429,...,0.000000,0.0,0.004063,0.0,0.000000,0.0,0.006281,0.000000,0.000000,0.000000,0.030391,0.000000,0.00000,0.027595,0.043131,0.0,0.011296,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.007418,0.01348,0.009529,0.000000,0.0,0.060063,0.0,0.0,0.000000,0.013191,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.029242,0.000000,0.000000,0.004271,0.0,0.0,0.000000,0.0,0.0,0.0
Grumpier Old Men,0.000000,0.049073,1.000000,0.0,0.025005,0.000000,0.00000,0.006500,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.018349,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.011910,0.0,0.010392,0.000000,0.006408,0.0,0.006683,0.000000,0.000000,0.005282,0.0,0.0,0.0,0.0,0.000000,0.000000,0.007480,0.0,0.000000,0.00000,0.016500,0.016821,0.000000,...,0.000000,0.0,0.000000,0.0,0.007415,0.0,0.000000,0.000000,0.000000,0.015443,0.000000,0.007561,0.00000,0.031215,0.000000,0.0,0.

## 영화 추천 함수 >> 유사도가 가장 비슷한 영화 10개

### df활용

In [36]:
def my_top10_movies(title, cosine_sim=cosine_sim):
    
    cos_sim_arr = df_cs.loc[title].sort_values(ascending = False)

    print(cos_sim_arr[1:11])  # 자기자신을 제외하기 위해서 [1:11] 슬라이싱


In [37]:
my_top10_movies("Toy Story")

title
Toy Story 3               0.525823
Toy Story 2               0.462639
The 40 Year Old Virgin    0.279554
The Champ                 0.200686
Rebel Without a Cause     0.182730
For Your Consideration    0.156892
Condorman                 0.152734
Man on the Moon           0.143199
Malice                    0.137511
Factory Girl              0.133607
Name: Toy Story, dtype: float64


### 교과서(비추비추)
* movie_dict = {"title":index}

In [38]:
# 검색을 위한 색인만들기
title_to_index = dict(zip(data['title'], data.index))
# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [39]:
data.loc[4,"title"]

'Father of the Bride Part II'

In [40]:
# def my_top10_movies(title, cosine_sim=cosine_sim):
#     # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
#     idx = title_to_index[title]

#     # 해당 영화와 모든 영화와의 유사도를 가져온다.
#     sim_scores = list(enumerate(cosine_sim[idx]))

#     # 유사도에 따라 영화들을 정렬한다.
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # 가장 유사한 10개의 영화를 받아온다.
#     sim_scores = sim_scores[1:11]

#     # 가장 유사한 10개의 영화의 인덱스를 얻는다.
#     movie_indices = [idx[0] for idx in sim_scores]

#     # 가장 유사한 10개의 영화의 제목을 리턴한다.
#     return data['title'].iloc[movie_indices]